# Libraries and data importing

Importing packages

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
from statsmodels.tsa.ar_model import AutoReg
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.statespace.structural import UnobservedComponents
from statsmodels.tsa.exponential_smoothing.ets import ETSModel

In [ ]:
from sklearn.metrics import r2_score, mean_absolute_error, root_mean_squared_error

Importing data

In [ ]:
activities = pd.read_parquet('../data-raw/activities.parquet')
activities

600 events at max for each contributor

In [ ]:
data = ( 
    activities
    # keep the last 600 events for each contributor
    .groupby('contributor')
    .tail(600)
    # keep the contributors who have more than 600 events
    .groupby('contributor')
    .filter(lambda x: len(x) == 600)
)
data

In [ ]:
def gap_activities(train, test):

    start_time = train['date'].iloc[-1] + pd.Timedelta(hours=1)
    end_time = test['date'].iloc[0] - pd.Timedelta(hours=1)

    #  check if there is a time gap between the train and test data
    if end_time - start_time >= pd.Timedelta(hours=0):

        # fill the gap with a date range and zeros for n_activities
        gap_data = pd.DataFrame({
            'category': train['category'].iloc[0],
            'date': pd.date_range(start=start_time, end=end_time, freq='H'),
            'contributor': train['contributor'].iloc[0],
            'n_activities': 0
        })

        test = pd.concat([gap_data, test]).reset_index(drop=True)
    
    return test

In [ ]:
def split_activities(contributor):

    # spliting the data into training and testing sets for time series forecasting, using a time-based split with split size = 0.5
    train, test = (
        contributor
        .apply(lambda x: x[:300]) # head(300)
        .groupby(['category', pd.Grouper(key='date', freq='H'), 'contributor'])['activity']
        .count()
        .reset_index(name='n_activities'),

        contributor
        .apply(lambda x: x[300:])
        .groupby(['category', pd.Grouper(key='date', freq='H'), 'contributor'])['activity']
        .count()
        .reset_index(name='n_activities')
    )

    # checking if the last timestamp of the train data is equal to the first timestamp of the second data
    if train['date'].iloc[-1] == test['date'].iloc[0]:
        # adding the value of the last time value (n_activities) of train data to the value of the first time (n_activities) of the test data
        test.loc[0, 'n_activities'] += train.loc[train.index[-1], 'n_activities']
        # removing the last time of the train data
        train.drop(train.index[-1], inplace=True)

    test = gap_activities(train, test)

    # filling n_activities with zeros for the empty hours between the minimum and maximum date
    train, test = (
        # for train set, we take last 3 months
        train[train['date'] >= train['date'].max() - pd.DateOffset(months=3)]
        .set_index('date')
        .resample('H')
        .sum()
        .rename_axis(None)
        .replace({'category': 0, 'contributor': 0}, None)
        .ffill(),
        
        test
        .set_index('date')
        .resample('H')
        .sum()
        .rename_axis(None)
        .replace({'category': 0, 'contributor': 0}, None)
        .ffill()
    )

    train.index.freq = 'H'

    return train, test

# New evaluation metrics PGA & CTD?

A new evaluation metric that calculates the percentage of predicted values greater than or equal to the actual values. We can define this metric as follows:

$$PGA = \frac{\sum_{i=1}^{n} [y_i \leq \hat{y}_i]}{n}$$

In [ ]:
def pga_score(y_true, y_pred):
    return (y_pred >= y_true).mean()

A novel evaluation metric designed to quantify the time difference between the cumulative sums of true and predicted values in reaching a specified target value.

$$ \text{CTD} = \text{argmax}(C_t \geq T) - \text{argmax}(C_p \geq T) $$

This formula represents the time difference between the cumulative sums of the true $C_t$ and predicted $C_p$ values in reaching a specified target value $T(100, 200, 300)$, where ${argmax}$ returns the time of the first occurrence where the condition is satisfied.

In [ ]:
def ctd_score(y_true, y_pred, target_value):

    coef = 1
    if (sum(y_true) < target_value) | (sum(y_pred) < target_value):
        coef = -1

    true_cumsum, pred_cumsum = np.cumsum(y_true), np.cumsum(y_pred)
    time_true, time_pred = np.argmax(true_cumsum >= target_value), np.argmax(pred_cumsum >= target_value)

    display(true_cumsum.tolist(), pred_cumsum.tolist())

    return coef*(time_true - time_pred)

In [ ]:
true_values = [2, 11, 84, 57, 0, 38, 15, 80, 4, 30, 90, 0, 0, 0]
pred_values = [52, 22, 95, 9, 11, 1, 73, 0, 30, 50, 100, 70, 50, 500]


print("Cumulative Time Difference:", ctd_score(true_values, pred_values, 400))

# 1. Autoregressive model

In [ ]:
def ar_model(contributor):

    print(contributor['contributor'].iloc[0])

    # Spliting the data into training and testing sets
    train, test = split_activities(contributor)

    # Fit the model
    try: #calculate parameter based on time range
        lags = [1, 12, 24, 168]
        model = AutoReg(train['n_activities'], lags=lags).fit()
        predictions = model.get_prediction(start=len(train), end=len(train)+len(test)-1).summary_frame(alpha=0.05)
    except IndexError:
        lags = [1, 12, 24]
        model = AutoReg(train['n_activities'], lags=lags).fit()
        predictions = model.get_prediction(start=len(train), end=len(train)+len(test)-1).summary_frame(alpha=0.05)
    except: # eviter
        lags = int(len(train)/2)-1
        model = AutoReg(train['n_activities'], lags=lags).fit()
        predictions = model.get_prediction(start=len(train), end=len(train)+len(test)-1).summary_frame(alpha=0.05)

    # Create a series for evaluation metrics and sum of activities
    metrics = pd.Series({
        'contributor': contributor['contributor'].iloc[0],
        'category': contributor['category'].iloc[0],
        'r2': r2_score(test['n_activities'], predictions['mean']),
        'mae': mean_absolute_error(test['n_activities'], predictions['mean']),
        'rmse': root_mean_squared_error(test['n_activities'], predictions['mean']),
        'pga': pga_score(test['n_activities'], predictions['mean']),
        'ctd_100': ctd_score(test['n_activities'], predictions['mean'], 100),
        'ctd_200': ctd_score(test['n_activities'], predictions['mean'], 200),
        'ctd_300': ctd_score(test['n_activities'], predictions['mean'], 300),
        'n_activities': train['n_activities'].sum(),
        'lags': lags,
        'true_values': test['n_activities'].values,
        'predicted_values': predictions['mean'].values,
    })

    return metrics

In [ ]:
# Apply the function to each contributor
ar_results = data.groupby(['category', 'contributor']).apply(ar_model).reset_index(drop=True)

In [ ]:
ar_results.head()

In [ ]:
ar_results.to_csv('../models-evaluation-v2/ar_model_metrics.csv', index=False)

# 2. Seasonal Autoregressive integrated Moving-average model

In [ ]:
def sarima_model(contributor):

    print(contributor['contributor'].iloc[0])

    # Spliting the data into training and testing sets
    train, test = split_activities(contributor)

    # Fit the model
    model = SARIMAX(train['n_activities'], order=(1, 1, 1), seasonal_order=(1, 1, 1, 24), enforce_invertibility=False, enforce_stationarity=False).fit(disp=False, method='lbfgs')

    # Forecast the test set using confidence interval with 95%
    predictions = model.get_prediction(start=len(train), end=len(train)+len(test)-1).summary_frame(alpha=0.05)

    # Create a series for evaluation metrics and sum of activities
    metrics = pd.Series({
        'contributor': contributor['contributor'].iloc[0],
        'category': contributor['category'].iloc[0],
        'r2': r2_score(test['n_activities'], predictions['mean']),
        'mae': mean_absolute_error(test['n_activities'], predictions['mean']),
        'rmse': root_mean_squared_error(test['n_activities'], predictions['mean']),
        'pga': pga_score(test['n_activities'], predictions['mean']),
        'ctd_100': ctd_score(test['n_activities'], predictions['mean'], 100),
        'ctd_200': ctd_score(test['n_activities'], predictions['mean'], 200),
        'ctd_300': ctd_score(test['n_activities'], predictions['mean'], 300),
        'n_activities': train['n_activities'].sum(),
        'true_values': test['n_activities'].values,
        'predicted_values': predictions['mean'].values,
    })

    return metrics

In [ ]:
# Apply the function to each contributor
sarima_results = data.groupby(['category', 'contributor']).apply(sarima_model).reset_index(drop=True)

In [ ]:
sarima_results.head()

In [ ]:
sarima_results['ctd_200'].describe()

In [ ]:
sarima_results.to_csv('../models-evaluation-v2/sarima_model_metrics.csv', index=False)

# 3. Unobserved components model

In [ ]:
def uc_model(contributor):

    print(contributor['contributor'].iloc[0])

    # Spliting the data into training and testing sets
    train, test = split_activities(contributor)

    # Fit the model
    model = UnobservedComponents(train['n_activities'], level=True, seasonal=24).fit(disp=False, method='lbfgs')

    # Forecast the test set using confidence interval with 95%
    predictions = model.get_prediction(start=len(train), end=len(train)+len(test)-1).summary_frame(alpha=0.05)

    # Create a series for evaluation metrics and sum of activities
    metrics = pd.Series({
        'contributor': contributor['contributor'].iloc[0],
        'category': contributor['category'].iloc[0],
        'r2': r2_score(test['n_activities'], predictions['mean']),
        'mae': mean_absolute_error(test['n_activities'], predictions['mean']),
        'rmse': root_mean_squared_error(test['n_activities'], predictions['mean']),
        'pga': pga_score(test['n_activities'], predictions['mean']),
        'ctd_100': ctd_score(test['n_activities'], predictions['mean'], 100),
        'ctd_200': ctd_score(test['n_activities'], predictions['mean'], 200),
        'ctd_300': ctd_score(test['n_activities'], predictions['mean'], 300),
        'n_activities': train['n_activities'].sum(),
        'true_values': test['n_activities'].values,
        'predicted_values': predictions['mean'].values,
    })

    return metrics

In [ ]:
# Apply the function to each contributor
uc_results = data.groupby(['category', 'contributor']).apply(uc_model).reset_index(drop=True)

In [ ]:
uc_results.head()

In [ ]:
uc_results[['contributor', 'category', 'r2', 'rmse', 'pga', 'ctd_100', 'ctd_200', 'ctd_300']].sample(10)

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Create a scatter plot
sns.scatterplot(data=uc_results, x='ctd_100', y='ctd_300', hue='category')
plt.xlabel('RMSE')
plt.ylabel('R2')
plt.title('RMSE vs. R2')
plt.show()

In [ ]:
uc_results['ctd_100'].describe()

In [ ]:
uc_results.to_csv('../models-evaluation-v2/uc_model_metrics.csv', index=False)

# 4. Holt-Winters (triple) exponential smoothing model

In [ ]:
def tes_model(contributor):

    print(contributor['contributor'].iloc[0])

    # Spliting the data into training and testing sets
    train, test = split_activities(contributor)

    # Fit the model
    try:
        model = ETSModel(train['n_activities'], error='add', trend='add', seasonal='add', seasonal_periods=24).fit(disp=False)
    except ValueError:
        model = ETSModel(train['n_activities'], error='add', trend='add').fit(disp=False)
    except:
        print("Something else went wrong")

    # Forecast the test set using prediction interval with 95%
    predictions = model.get_prediction(start=len(train), end=len(train)+len(test)-1).summary_frame(alpha=0.05)

    # Create a series for evaluation metrics and sum of activities
    metrics = pd.Series({
        'contributor': contributor['contributor'].iloc[0],
        'category': contributor['category'].iloc[0],
        'r2': r2_score(test['n_activities'], predictions['mean']),
        'mae': mean_absolute_error(test['n_activities'], predictions['mean']),
        'rmse': root_mean_squared_error(test['n_activities'], predictions['mean']),
        'pga': pga_score(test['n_activities'], predictions['mean']),
        'ctd_100': ctd_score(test['n_activities'], predictions['mean'], 100),
        'ctd_200': ctd_score(test['n_activities'], predictions['mean'], 200),
        'ctd_300': ctd_score(test['n_activities'], predictions['mean'], 300),
        'n_activities': train['n_activities'].sum(),
        'true_values': test['n_activities'].values,
        'predicted_values': predictions['mean'].values,
    })

    return metrics

In [ ]:
# Apply the function to each contributor
tes_results = data.groupby(['category', 'contributor']).apply(tes_model).reset_index(drop=True)

In [ ]:
tes_results.head()

In [ ]:
tes_results['ctd_100'].describe()

In [ ]:
tes_results.to_csv('../models-evaluation-v2/tes_model_metrics.csv', index=False)

# Models comparing

In [ ]:
ar_results = pd.read_csv('../models-evaluation-v2/ar_model_metrics.csv')
sarima_results = pd.read_csv('../models-evaluation-v2/sarima_model_metrics.csv')
uc_results = pd.read_csv('../models-evaluation-v2/uc_model_metrics.csv')
tes_results = pd.read_csv('../models-evaluation-v2/tes_model_metrics.csv')

In [ ]:
ar_results['model'] = 'ar'
sarima_results['model'] = 'sarima'
uc_results['model'] = 'uc'
tes_results['model'] = 'tes'

In [ ]:
ar_results.drop('lags', axis=1, inplace=True)

In [ ]:
results = pd.concat([ar_results, sarima_results, uc_results, tes_results], ignore_index=True)
results.drop(['n_activities', 'true_values', 'predicted_values'], axis=1, inplace=True)

results['ctd_100_abs'] = results['ctd_100'].abs()
results['ctd_200_abs'] = results['ctd_200'].abs()
results['ctd_300_abs'] = results['ctd_300'].abs()

In [ ]:
sns.set(style="whitegrid")

fig, axes = plt.subplots(1, 3, figsize=(14, 5))

sns.boxenplot(data=results, x='model', y='ctd_100', hue='category', ax=axes[0], showfliers=False)
sns.boxenplot(data=results, x='model', y='ctd_200', hue='category', ax=axes[1], showfliers=False)
sns.boxenplot(data=results, x='model', y='ctd_300', hue='category', ax=axes[2], showfliers=False)

plt.tight_layout()
plt.show()

In [ ]:
sarima_results

In [ ]:
sns.scatterplot(data=sarima_results.assign(nh=lambda d: d.true_values.apply(len)), x='ctd_300', y='nh', hue='category')

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(14, 5))

sns.boxenplot(data=results, x='model', y='mae', hue='category', ax=axes[0], showfliers=False)
sns.boxenplot(data=results, x='model', y='rmse', hue='category', ax=axes[1], showfliers=False)
sns.boxenplot(data=results, x='model', y='pga', hue='category', ax=axes[2], showfliers=False)

axes[0].set_ylim(0, 15)
axes[1].set_ylim(0, 15)

plt.tight_layout()
plt.show()

# Activitiy types

In [ ]:
random_user = data['contributor'].sample().values[0]

contributor = (
    data[data['contributor'] == random_user]
    .reset_index(drop=True)[['activity', 'date']]
    .groupby([pd.Grouper(key='date', freq='D'), 'activity'])['activity']
    .count()
    .reset_index(name='n_activities')
    .set_index('date')
    .rename_axis(None)
    .pivot(columns='activity', values='n_activities')
    .fillna(0)
    .astype(int)
)
contributor.insert(0, 'n_activities', contributor.sum(axis=1))

plt.figure(figsize=(16, 6))
sns.lineplot(data=contributor, palette="tab10")
plt.xlabel('Timestamp')
plt.ylabel('Number of Activities')
plt.title('Activities Over Time for {}'.format(random_user))
plt.xticks(rotation=45)
plt.show()